# ETL Processes

Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import glob
import psycopg2
import pandas as pd
import logging
import warnings

from rich import traceback
from pathlib import Path


In [3]:
src_path: str = "../src"
sys.path.append(src_path)
_ = traceback.install()
logging.basicConfig(force=True)
logging.getLogger().setLevel(logging.ERROR)
warnings.filterwarnings("ignore")


In [4]:
from sql_queries import *
from create_tables import main as run_create_tables
from etl import main as run_etl

run_create_tables()


In [5]:
conn = psycopg2.connect(
    "host=127.0.0.1 dbname=sparkifydb user=student password=student"
)
cur = conn.cursor()


In [6]:
song_data_path = Path("../data/song_data")
log_data_path = Path("../data/log_data")


In [7]:
def get_files(filepath: Path, file_pattern: str = "**/*.json"):
    return [f.resolve() for f in filepath.glob(file_pattern)]


# Process `song_data`

In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.

- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data


In [8]:
song_files = get_files(song_data_path)
song_files[:5]


[PosixPath('/home/uziel/Development/sparkify/data/song_data/A/B/C/TRABCEC128F426456E.json'),
 PosixPath('/home/uziel/Development/sparkify/data/song_data/A/B/C/TRABCUQ128E0783E2B.json'),
 PosixPath('/home/uziel/Development/sparkify/data/song_data/A/B/C/TRABCKL128F423A778.json'),
 PosixPath('/home/uziel/Development/sparkify/data/song_data/A/B/C/TRABCFL128F149BB0D.json'),
 PosixPath('/home/uziel/Development/sparkify/data/song_data/A/B/C/TRABCYE128F934CE1D.json')]

In [9]:
song_meta = pd.read_json(song_files[0], typ="series").rename(song_files[0].stem)
song_meta


num_songs                            1
artist_id           AR0IAWL1187B9A96D0
artist_latitude                 8.4177
artist_longitude             -80.11278
artist_location                 Panama
artist_name               Danilo Perez
song_id             SONSKXP12A8C13A2C9
title                      Native Soul
duration                     197.19791
year                              2003
Name: TRABCEC128F426456E, dtype: object

## #1: `songs` Table

#### Extract Data for Songs Table

- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`


In [10]:
song_columns = ["song_id", "title", "artist_id", "year", "duration"]
song_data = song_meta[song_columns]
song_data


song_id      SONSKXP12A8C13A2C9
title               Native Soul
artist_id    AR0IAWL1187B9A96D0
year                       2003
duration              197.19791
Name: TRABCEC128F426456E, dtype: object

#### Insert Record into Song Table

Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.


In [11]:
cur.execute(get_insert_query("songs", song_data.index), song_data.values)
conn.commit()


Run `test.ipynb` to see if you've successfully added a record to this table.


## #2: `artists` Table

#### Extract Data for Artists Table

- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`


In [12]:
artist_columns = [
    "artist_id",
    "artist_name",
    "artist_location",
    "artist_latitude",
    "artist_longitude",
]
artist_data = song_meta[artist_columns]
artist_data


artist_id           AR0IAWL1187B9A96D0
artist_name               Danilo Perez
artist_location                 Panama
artist_latitude                 8.4177
artist_longitude             -80.11278
Name: TRABCEC128F426456E, dtype: object

#### Insert Record into Artist Table

Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.


In [13]:
cur.execute(
    get_insert_query(
        "artists", ("artist_id", "name", "location", "latitude", "longitude")
    ),
    artist_data.values,
)
conn.commit()


Run `test.ipynb` to see if you've successfully added a record to this table.


# Process `log_data`

In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.

- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data


In [14]:
log_files = get_files(log_data_path)
log_files[:5]


[PosixPath('/home/uziel/Development/sparkify/data/log_data/2018/11/2018-11-13-events.json'),
 PosixPath('/home/uziel/Development/sparkify/data/log_data/2018/11/2018-11-29-events.json'),
 PosixPath('/home/uziel/Development/sparkify/data/log_data/2018/11/2018-11-01-events.json'),
 PosixPath('/home/uziel/Development/sparkify/data/log_data/2018/11/2018-11-20-events.json'),
 PosixPath('/home/uziel/Development/sparkify/data/log_data/2018/11/2018-11-14-events.json')]

In [15]:
log_data = pd.read_json(log_files[0], lines=True)
log_data.head()


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Kevin,M,0,Arellano,NaN,free,"Harrisburg-Carlisle, PA",GET,Home,1.540007e+12,514,None,200,1542069417796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
1,Fu,Logged In,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,514,Ja I Ty,200,1542069637796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
2,None,Logged In,Maia,F,0,Burke,NaN,free,"Houston-The Woodlands-Sugar Land, TX",GET,Home,1.540677e+12,510,None,200,1542071524796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",51
3,All Time Low,Logged In,Maia,F,1,Burke,177.84118,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540677e+12,510,A Party Song (The Walk of Shame),200,1542071549796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",51
4,Nik & Jay,Logged In,Wyatt,M,0,Scott,196.51873,free,"Eureka-Arcata-Fortuna, CA",PUT,NextSong,1.540872e+12,379,Pop-Pop!,200,1542079142796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9


## #3: `time` Table

#### Extract Data for Time Table

- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe


In [16]:
log_data = log_data[log_data["page"] == "NextSong"]
log_data.head()


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Fu,Logged In,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,514,Ja I Ty,200,1542069637796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
3,All Time Low,Logged In,Maia,F,1,Burke,177.84118,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540677e+12,510,A Party Song (The Walk of Shame),200,1542071549796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",51
4,Nik & Jay,Logged In,Wyatt,M,0,Scott,196.51873,free,"Eureka-Arcata-Fortuna, CA",PUT,NextSong,1.540872e+12,379,Pop-Pop!,200,1542079142796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9
5,Quad City DJ's,Logged In,Chloe,F,0,Cuevas,451.44771,free,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,506,C'mon N' Ride It (The Train) (LP Version),200,1542081112796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
8,Hoobastank,Logged In,Noah,M,1,Chavez,232.17587,free,"Ogden-Clearfield, UT",PUT,NextSong,1.540971e+12,492,Born To Lead,200,1542085206796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) G...,94


In [17]:
log_data["ts"] = pd.to_datetime(log_data["ts"], unit="ms")
log_data.head()


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Fu,Logged In,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,514,Ja I Ty,200,2018-11-13 00:40:37.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
3,All Time Low,Logged In,Maia,F,1,Burke,177.84118,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540677e+12,510,A Party Song (The Walk of Shame),200,2018-11-13 01:12:29.796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",51
4,Nik & Jay,Logged In,Wyatt,M,0,Scott,196.51873,free,"Eureka-Arcata-Fortuna, CA",PUT,NextSong,1.540872e+12,379,Pop-Pop!,200,2018-11-13 03:19:02.796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9
5,Quad City DJ's,Logged In,Chloe,F,0,Cuevas,451.44771,free,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,506,C'mon N' Ride It (The Train) (LP Version),200,2018-11-13 03:51:52.796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
8,Hoobastank,Logged In,Noah,M,1,Chavez,232.17587,free,"Ogden-Clearfield, UT",PUT,NextSong,1.540971e+12,492,Born To Lead,200,2018-11-13 05:00:06.796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) G...,94


In [18]:
column_labels = ("hour", "day", "week", "month", "year", "dayofweek")


In [19]:
time_df = (
    pd.DataFrame(
        {ts: {col: getattr(ts, col) for col in column_labels} for ts in log_data["ts"]}
    )
    .transpose()
    .reset_index()
    .rename(columns={"index": "start_time", "dayofweek": "weekday"})
)
time_df.head()


,start_time,hour,day,week,month,year,weekday
0,2018-11-13 00:40:37.796,0,13,46,11,2018,1
1,2018-11-13 01:12:29.796,1,13,46,11,2018,1
2,2018-11-13 03:19:02.796,3,13,46,11,2018,1
3,2018-11-13 03:51:52.796,3,13,46,11,2018,1
4,2018-11-13 05:00:06.796,5,13,46,11,2018,1


#### Insert Records into Time Table

Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.


In [20]:
for _, row in time_df.iterrows():
    cur.execute(get_insert_query("time", row.index), row.values)
    conn.commit()


Run `test.ipynb` to see if you've successfully added records to this table.


## #4: `users` Table

#### Extract Data for Users Table

- Select columns for user ID, first name, last name, gender and level and set to `user_df`


In [21]:
json_db_cols_map = {
    "userId": "user_id",
    "firstName": "first_name",
    "lastName": "last_name",
    "gender": "gender",
    "level": "level",
}


In [22]:
user_df = (
    log_data[json_db_cols_map.keys()]
    .rename(columns=json_db_cols_map)
    .drop_duplicates("user_id")
)
user_df.head()


,user_id,first_name,last_name,gender,level
1,66,Kevin,Arellano,M,free
3,51,Maia,Burke,F,free
4,9,Wyatt,Scott,M,free
5,49,Chloe,Cuevas,F,free
8,94,Noah,Chavez,M,free


#### Insert Records into Users Table

Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.


In [23]:
for _, row in user_df.iterrows():
    cur.execute(get_insert_query("users", row.index), row.values)
    conn.commit()


Run `test.ipynb` to see if you've successfully added records to this table.


## #5: `songplays` Table

#### Extract Data and Songplays Table

This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.

- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table

- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.


In [24]:
songplay_cols = (
    "songplay_id",
    "start_time",
    "user_id",
    "level",
    "song_id",
    "artist_id",
    "session_id",
    "location",
    "user_agent",
)

for index, row in log_data.iterrows():
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()

    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (
        index,
        row["ts"],
        row["userId"],
        row["level"],
        songid,
        artistid,
        row["sessionId"],
        row["location"],
        row["userAgent"],
    )
    cur.execute(get_insert_query("songplays", songplay_cols), songplay_data)
    conn.commit()


Run `test.ipynb` to see if you've successfully added records to this table.


# Close Connection to Sparkify Database


In [25]:
conn.close()


# Implement `etl.py`

Use what you've completed in this notebook to implement `etl.py`.


In [26]:
run_etl()


Output()

Output()